In [4]:
import os
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    embedding_weights_path: Path
    params_num_encoder_layers: int
    params_num_tags: int
    params_vocab_size: int
    params_embedding_dim: int
    params_fully_connected_dim: int
    params_num_heads: int
    params_max_positional_encoding_length: int
    params_epochs: int
    params_batch_size: int



In [6]:
from named_entity_recognition.constants import *
from named_entity_recognition.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = Path(config.root_dir),
            data_path = Path(config.data_path),
            embedding_weights_path=Path(config.embedding_weights_path),
            params_num_encoder_layers=params.NUM_ENCODER_LAYERS,
            params_num_tags = params.NUM_TAGS,
            params_vocab_size = params.VOCAB_SIZE,
            params_embedding_dim = params.EMBEDDING_DIM,
            params_fully_connected_dim = params.FULLY_CONNECTED_DIM,
            params_num_heads = params.NUM_HEADS,
            params_max_positional_encoding_length = params.MAX_POSITIONAL_ENCODING_LENGTH,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE
        )

        return model_trainer_config

In [8]:
import os
import pickle
from datasets import load_from_disk
from named_entity_recognition import logger
from named_entity_recognition.transformer.model import NERModel
from named_entity_recognition.transformer.optimizer import get_optimizer_with_custom_lr_sheduler
from named_entity_recognition.transformer.losses import MaskedLoss
from named_entity_recognition.transformer.metrics import masked_acc
import numpy as np
import tensorflow as tf

c:\Users\sajit\anaconda3\envs\tf(GPU)3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelTrainer:

    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def get_model(self):

        model = NERModel(
            num_tags=self.config.params_num_tags,
            num_layers=self.config.params_num_encoder_layers,
            embedding_dim=self.config.params_embedding_dim,
            fully_connected_dim=self.config.params_fully_connected_dim,
            num_heads=self.config.params_num_heads,
            vocab_size=self.config.params_vocab_size,
            max_positional_encoding=self.config.params_max_positional_encoding_length
        )

        self.model = model
        self.model.encoder.embedding.build((1,))
        self.model.encoder.embedding.set_weights([self.load_embedding_weights()])
        self.model.encoder.embedding.trainable = False

        optimizer = get_optimizer_with_custom_lr_sheduler(embedding_dim=self.config.params_embedding_dim)
        
        self.model.compile(optimizer=optimizer, loss=MaskedLoss(), metrics=[masked_acc])

    def train(self):

        data = load_from_disk(str(self.config.data_path))
        early_stopping = tf.keras.callbacks.EarlyStopping(
                                        monitor='val_loss',
                                        patience=3,
                                        restore_best_weights=True,
                                    )
        
        train = tf.data.Dataset.from_tensor_slices((data['train']['input_ids'],data['train']['attention_mask'],data['train']['labels'])).map(lambda x,y,z: (tf.concat((x,y),axis=-1),z)).shuffle(10000).batch(self.config.params_batch_size)
        val = tf.data.Dataset.from_tensor_slices((data['validation']['input_ids'],data['validation']['attention_mask'],data['validation']['labels'])).map(lambda x,y,z: (tf.concat((x,y),axis=-1),z)).batch(self.config.params_batch_size)

        self.model.fit(
            train,
            validation_data=(val),
            epochs=self.config.params_epochs,
            callbacks=[early_stopping]
        )
        

        self.save_model(os.path.join(self.config.root_dir,'model.keras'),self.model)
        


    @staticmethod
    def save_model(path: Path, model: tf.keras.models.Model):
        model.save(path)

    def load_tokenizer(self):
        with open(self.config.tokenizer_path, 'rb') as f:
            tokenizer = pickle.load(f)
        return tokenizer
    
    def load_embedding_weights(self):
        with open(self.config.embedding_weights_path, 'rb') as f:
            embedding_weights = pickle.load(f)
        return embedding_weights

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.get_model()
    model_trainer.train()

except Exception as e:
    raise e

[2024-04-25 01:20:28,648: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-25 01:20:28,650: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-25 01:20:28,651: INFO: common: created directory at: artifacts]
[2024-04-25 01:20:28,653: INFO: common: created directory at: artifacts/model_trainer]
Epoch 1/10
 96/220 [============>.................] - ETA: 7s - loss: 1.7601 - masked_acc: 0.4489

KeyboardInterrupt: 